## Chargement des historiques Breeze
Ce notebook charge automatiquement les fichiers CSV générés par `breeze-historical-options` afin de préparer les données pour l'analyse.


In [4]:
!pip install ib_insync


   ---------------------------------------- 0/2 [eventkit]
   ---------------------------------------- 0/2 [eventkit]
   -------------------- ------------------- 1/2 [ib_insync]
   -------------------- ------------------- 1/2 [ib_insync]
   -------------------- ------------------- 1/2 [ib_insync]
   ---------------------------------------- 2/2 [ib_insync]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\romai\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 10061] Connect call failed ('127.0.0.1', 7497)

In [1]:
from pathlib import Path
from typing import Iterable, Optional

import pandas as pd

DATA_ROOT = Path("breeze-historical-options") / "HistoricData"


def list_csv_files(root: Path) -> list[Path]:
    if not root.exists():
        raise FileNotFoundError(
            f"Le dossier {root} est introuvable. Lance `example.py` pour créer les données."
        )
    csv_files = sorted(root.rglob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(
            f"Aucun CSV trouvé sous {root}. Vérifie les paramètres d'export Breeze."
        )
    return csv_files


def load_breeze_csv(files: Iterable[Path], limit: Optional[int] = None) -> pd.DataFrame:
    frames = []
    for idx, path in enumerate(files):
        if limit is not None and idx >= limit:
            break
        # Structure attendue: HistoricData/<scrip>/<year>/<month>/<expiry>_expiry/<strike>_<type>.csv
        try:
            scrip, year, month, expiry_folder, file_name = path.parts[-5:]
            strike, option_type = file_name.replace(".csv", "").split("_")
        except ValueError as exc:
            raise ValueError(f"Chemin inattendu pour {path}") from exc

        df = pd.read_csv(path)
        df["scrip"] = scrip
        df["expiry_year"] = int(year)
        df["expiry_month"] = month
        df["expiry_date"] = expiry_folder.replace("_expiry", "")
        df["strike"] = int(strike)
        df["option_type"] = option_type
        frames.append(df)

    if not frames:
        raise ValueError("Aucun fichier n'a été chargé (peut-être à cause du paramètre limit).")

    return pd.concat(frames, ignore_index=True)



In [2]:
csv_files = list_csv_files(DATA_ROOT)
print(f"{len(csv_files)} fichiers trouvés sous {DATA_ROOT}")

# Ajuste `limit` si tu veux échantillonner un nombre restreint de fichiers.
limit = None
data = load_breeze_csv(csv_files, limit=limit)

data.info()
data.head()

FileNotFoundError: Le dossier breeze-historical-options\HistoricData est introuvable. Lance `example.py` pour créer les données.